In [ ]:
import numpy
from keras.datasets import mnist
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import AveragePooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers
from keras.utils import plot_model
from matplotlib import pyplot as plt
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
K.set_image_dim_ordering('th')

x = np.loadtxt("train_x.csv", delimiter=",")  # load from text
y = np.loadtxt("train_y.csv", delimiter=",")
z = np.loadtxt("test_x.csv", delimiter=",")
x = x.reshape(-1, 64 * 64).astype('float32')  # reshape
y = y.reshape(-1).astype('float32')
z = z.reshape(-1, 64 * 64).astype('float32')
x_test = x[:220]
y_test = y [:220]
x_train = x[:-5000]
y_train = y[:-5000]
    #validate set
x_validate = x[-5000:]
y_validate = y[-5000:]
unique_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 54, 56, 63, 64, 72, 81]

def one_hot_encoding(y):
    encoded_y = []
    N = len(y)
    for i in range(N):
        encoded_y.append(unique_classes.index(int(y[i])))
    return encoded_y

# fix random seed for reproducibility
seed = 3
numpy.random.seed(seed)

# load data
# reshape to be [samples][pixels][width][height]
X_train = x_train.reshape(-1, 1, 64, 64).astype('float32')
X_test = x_validate.reshape(-1, 1, 64, 64).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(one_hot_encoding(y_train))
y_test = np_utils.to_categorical(one_hot_encoding(y_validate))
num_classes = y_test.shape[1]

# define the larger model
def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(120, (5, 5),strides=2, input_shape=(1, 64, 64), activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Conv2D(60, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Conv2D(30, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = larger_model()
model.summary()

Using TensorFlow backend.


In [ ]:
filepath="./model/weights-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=100,verbose=2,callbacks=callbacks_list)#,callbacks=callbacks_list
scores = model.evaluate(X_test, y_test, verbose=0)
prediction = model.predict_classes(X_test.astype('float32'))
print(prediction)
prediction = model.predict_classes(z.reshape(-1, 1, 64, 64).astype('float32'))
with open ('result.txt','w') as result:
    result.write("Id,Label\n")
    id = 1
    for i in prediction:
        result.write(str(id) + "," + str(i) + "\n")
        id += 1
print("CNN Error: %.2f%%" % (100-scores[1]*100))
history_dict = history.history
history_dict.keys()
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'g^')
plt.plot(epochs, val_loss_values, 'bs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()